In [1]:
#Code to get multiple outputs in the same cell

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

#Code to ignore all warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore',category=DeprecationWarning)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data=pd.read_csv(r"C:\Users\97158\Desktop\Apziva\Project 3- Potential Talents\potential-talents - Aspiring human resources - seeking human resources.csv")
data.head()
data1=data.copy()
orig_data=data.copy()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [4]:
#Importing relevant libraries

import pickle
from collections import Counter

import nltk
from nltk.corpus import stopwords
from nltk. stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [5]:
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# Functions to clean the text in the dataframe


# Function to to make all text lowercase
def make_lowercase(text):
    #lower=list(text)
    lower= [doc.lower() for doc in text]
    return lower

#Removing numbers
def remove_numbers(text):
    remover=[[re.sub(r'\d+','',word) for word in doc] for doc in text]
    #remover=re.sub(r'\d+','',text) # \d+ removes numbers. re.sub finds and replaces that pattern in text. Takes 3 arguments 
    return remover

#Removing punctuations
regex=RegexpTokenizer(r'\w+')
def remove_punctuations(text):
    removed_punct= [["".join(regex.tokenize(word)) for word in doc if len(regex.tokenize(word))>0] for doc in text]  
    return removed_punct

#Removing Stop Words
stop_words=stopwords.words('english')
def remove_stopwords(text):
    sw_remover=[[word for word in doc if word not in stop_words] for doc in text]
    return sw_remover


#Tokenizing texts
def tokenize(text):
    tokens=[word_tokenize(doc) for doc in text]
    return tokens

#Lemmatize Words
lemmatizer=WordNetLemmatizer()
def root(text):
    #flattened_text=[word for sublist in text for word in sublist]
    #lemmatized_text=[lemmatizer.lemmatize(word) for word in flattened_text]
    # text= [lemmatizer.lemmatize(word) for word_list in text for word in word_list]
    lem_text= [[lemmatizer.lemmatize(word) for word in doc] for doc in text]
    return lem_text


#Compiling all the above def functions to preprocess all in one step

def preprocess(text):

    text=make_lowercase(text)
    text=tokenize(text)
    text=remove_stopwords(text)
    text=remove_punctuations(text)
    text=remove_numbers(text)
    text=root(text)
  
    return text

In [7]:
data['job_title']=preprocess(data['job_title'])
data['location']=preprocess(data['location'])

In [8]:
data.head()

,id,job_title,location,connection,fit
0,1,"[, ct, bauer, college, business, graduate, mag...","[houston, texas]",85,NaN
1,2,"[native, english, teacher, epik, english, prog...",[kanada],500+,NaN
2,3,"[aspiring, human, resource, professional]","[raleighdurham, north, carolina, area]",44,NaN
3,4,"[people, development, coordinator, ryan]","[denton, texas]",500+,NaN
4,5,"[advisory, board, member, celal, bayar, univer...","[izmir, türkiye]",500+,NaN


In [9]:
flat_tokens=[word for t in data['job_title'] for word in t]
counts=Counter(flat_tokens)

counts.most_common(10)
len(counts.keys())

[('human', 63),
 ('resource', 63),
 ('aspiring', 35),
 ('professional', 21),
 ('student', 16),
 ('seeking', 15),
 ('college', 14),
 ('generalist', 14),
 ('university', 12),
 ('specialist', 12)]

178

# BOW Model

In [10]:
Data_bow=orig_data.copy()

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
BOW_Vectorizer=CountVectorizer()

In [13]:
Data_BOW=data.copy()
Data_BOW['job_title'] = Data_BOW['job_title'].apply(lambda x: ', '.join(x))

In [14]:
Query= "Aspiring human resources"

# We will first fit the current data (job_title) in the model and then seek to transofrm 
# the Query data to obtain consistent vector shape

job_vec=BOW_Vectorizer.fit_transform(Data_BOW['job_title'])
Query_vec=BOW_Vectorizer.transform([Query])


Query_vec.shape
job_vec.shape

(1, 176)

(104, 176)

In [15]:
def Sim_Score_Cal(Query_vec,raw_data_vectors,data,Top_n):
    
    x = []
    for i in range(raw_data_vectors.shape[0]):
        x.append(raw_data_vectors[i])

   
    data['fit']=x
    data['sim_score']=data['fit'].apply(lambda x: cosine_similarity(Query_vec,x).flatten())
    data=data.sort_values(by='sim_score', ascending=False)
    data=data.drop_duplicates(subset=['job_title'])
    data=data.drop('fit',axis=1)
    return  data.iloc[:Top_n]

In [16]:
Sim_Score_Cal(Query_vec,job_vec,Data_bow,10)

,id,job_title,location,connection,sim_score
48,49,Aspiring Human Resources Specialist,Greater New York City Area,1,[0.7071067811865475]
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,[0.7071067811865475]
72,73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,[0.6123724356957945]
38,39,Student at Humber College and Aspiring Human R...,Kanada,61,[0.5345224838248487]
28,29,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,[0.5345224838248487]
71,72,Business Management Major and Aspiring Human R...,"Monroe, Louisiana Area",5,[0.5345224838248487]
65,66,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57,[0.5345224838248487]
81,82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,[0.5345224838248487]
78,79,Liberal Arts Major. Aspiring Human Resources A...,"Baton Rouge, Louisiana Area",7,[0.5345224838248487]
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,[0.5144957554275265]


# TF-IDF

In [17]:
Data_tfidf=orig_data.copy()

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
Vectorizer=TfidfVectorizer()

In [20]:
Data_TFIDF=data.copy()
Data_TFIDF['job_title'] = Data_TFIDF['job_title'].apply(lambda x: ', '.join(x))

In [21]:
Query= "Aspiring human resources"

# We will first fit the current data (job_title) in the model and then seek to transofrm 
# the Query data to obtain consistent vector shape

job_vec=Vectorizer.fit_transform(Data_TFIDF['job_title'])
Query_vec=Vectorizer.transform([Query])


print('Tfidf_Transformed_Query shape: ' ,Query_vec.shape)
print('Tfidf_Transformed_Data Shape: ' ,job_vec.shape)

Tfidf_Transformed_Query shape:  (1, 176)
Tfidf_Transformed_Data Shape:  (104, 176)


In [22]:
def Sim_Score_Cal(Query_vec,raw_data_vectors,data,Top_n):
    
    x = []
    for i in range(raw_data_vectors.shape[0]):
        x.append(raw_data_vectors[i])

   
    data['fit']=x
    data['sim_score']=data['fit'].apply(lambda x: cosine_similarity(Query_vec,x).flatten())
    data=data.sort_values(by='sim_score', ascending=False)
    data=data.drop_duplicates(subset=['job_title'])
    data=data.drop('fit',axis=1)
    return  data.iloc[:Top_n]

In [23]:
Sim_Score_Cal(Query_vec,job_vec,Data_tfidf,10)

,id,job_title,location,connection,sim_score
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,[0.6530518413030217]
48,49,Aspiring Human Resources Specialist,Greater New York City Area,1,[0.5982454662804461]
72,73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,[0.44098511935836665]
38,39,Student at Humber College and Aspiring Human R...,Kanada,61,[0.375084122551786]
26,27,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,[0.3402313811852311]
71,72,Business Management Major and Aspiring Human R...,"Monroe, Louisiana Area",5,[0.32224246587670147]
65,66,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57,[0.2997489634099375]
81,82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,[0.2782324858813505]
73,74,Human Resources Professional,Greater Boston Area,16,[0.2704020827550762]
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,[0.2594028839742114]


# Word2Vec

In [24]:
Data_Word2Vec=data.copy()

In [25]:
# Training Skip Gram Model for Word2Vec

skipgram2=Word2Vec(Data_Word2Vec['job_title'],vector_size=100,window=2,min_count=2,sg=1)

#Window=3 means model will consider 1 word at a time while training
#min_count=2 means that words that appear at least twice will be included in the vocab.

print(skipgram2)

Word2Vec<vocab=67, vector_size=100, alpha=0.025>


In [26]:
# Model Embeddings-Similarity

skipgram2.wv.most_similar(positive=['human'], topn=10)

[('marketing', 0.2013355791568756),
 ('manager', 0.19968749582767487),
 ('hr', 0.19095101952552795),
 ('bauer', 0.18173237144947052),
 ('humber', 0.16728708148002625),
 ('english', 0.16227439045906067),
 ('america', 0.1455521583557129),
 ('development', 0.14216649532318115),
 ('svp', 0.12910626828670502),
 ('ct', 0.12734930217266083)]

In [27]:
# Function to take average of all vectors of each word in job_title so that each job_title can represent (1,100) dimensional vector

def get_mean_vector(Model, doc):

    words=[word for word in doc if word in Model.wv.index_to_key]
    if len(words)>=1:
        return np.mean(Model.wv[words], axis=0)
    else:
        return np.array([0]*100)

In [28]:
# # Function to take average of a vector for each word in job_title so that each job_title can represt (1,100) dimensinal vector
# def query_mean(Model, query):

#     words=[word for word in query if word in Model.wv.index_to_key]
#     if len(query)>=1:
#         return np.mean(Model.wv[query], axis=0)
#     else:
#         return np.array([0]*100)

In [29]:
# Job_title to Vectors using Skip Gram Model

sg_mean= [get_mean_vector(skipgram2,i) for i in data.job_title]
skipg_mean=pd.DataFrame(sg_mean)

skipg_mean.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.001345,0.001193,0.002517,0.002969,0.000618,-0.002404,8.835207e-07,0.004786,-0.002561,-0.002443,...,0.002675,-0.000027,0.000926,0.001033,0.001884,0.002226,0.000260,-0.001268,-0.000932,-0.000929
1,-0.002065,-0.001675,-0.002731,0.000154,-0.001379,-0.003207,-3.798845e-03,-0.001079,-0.000762,-0.002693,...,0.002442,0.002448,0.002155,-0.001035,-0.000679,-0.000289,0.001786,-0.002439,-0.001851,-0.000201
2,-0.004331,0.003942,0.000803,0.002981,0.002465,-0.002866,1.598239e-03,0.006624,-0.002718,-0.003187,...,-0.001884,0.000751,0.003638,-0.003435,0.006241,0.003303,0.001994,-0.003863,-0.000977,-0.000495
3,-0.004154,-0.003903,0.003119,-0.003799,-0.002202,-0.000479,1.790616e-03,0.004054,-0.000794,-0.002236,...,0.004498,-0.006257,0.000074,-0.005798,-0.001003,-0.002423,0.008033,0.001198,0.004467,0.002589
4,-0.002185,-0.000136,0.000374,0.002807,-0.002082,-0.004179,5.893023e-04,0.002730,-0.001350,-0.003703,...,0.002409,0.002446,-0.000637,0.002519,0.005130,-0.001134,-0.006044,0.002324,-0.001782,0.000846


In [30]:
skip_gram_vectors=skipg_mean.to_numpy()
print('Word2Vec_Shape: ',skip_gram_vectors.shape)


Word2Vec_Shape:  (104, 100)


In [31]:
def Input_preprocessor(model, query):
    if len(query)>=1:
        return model.wv[query].reshape(1,-1)
    else:
        return np.array([0]*100)

In [32]:
def Top_n(query, model, vectors,data,Top_n):

    query = Input_preprocessor(model, query)
    
    x = []
    for i in range(len(vectors)):
        x.append(cosine_similarity(query,vectors[i].reshape(1,-1)).flatten())
        
    data['fit']=x
    data=data.sort_values(by='fit', ascending=False)
    data=data.drop_duplicates(subset=['job_title'])
    data=data.iloc[:Top_n]
    return  data

In [33]:
Query= "Aspiring human resources"
Q_df=pd.DataFrame([Query])
Top_n(Q_df, skipgram2,skip_gram_vectors,Data_Word2Vec,10)

,id,job_title,location,connection,fit
76,77,"[human, resource, conflict, management, policy...","[dallasfort, worth, area]",409,[0.6677329]
87,88,"[human, resource, management, major]","[milpitas, california]",18,[0.6276028]
77,78,"[human, resource, generalist, schwan, s]","[amerika, birleşik, devletleri]",500+,[0.61696106]
70,71,"[human, resource, generalist, scottmadden, inc]","[raleighdurham, north, carolina, area]",500+,[0.61696106]
100,101,"[human, resource, generalist, loparex]","[raleighdurham, north, carolina, area]",500+,[0.61696106]
67,68,"[human, resource, specialist, luxottica]","[greater, new, york, city, area]",500+,[0.5867844]
88,89,"[director, human, resource, ey]","[greater, atlanta, area]",349,[0.57659316]
72,73,"[aspiring, human, resource, manager, seeking, ...","[houston, texas, area]",7,[0.5688066]
78,79,"[liberal, art, major, aspiring, human, resourc...","[baton, rouge, louisiana, area]",7,[0.5447279]
99,100,"[aspiring, human, resource, manager, graduatin...","[cape, girardeau, missouri]",103,[0.5411712]


 # FastText

In [34]:
Data_FastText=data.copy()

In [35]:
# Training FastText Model

fast_n= FastText(Data_FastText['job_title'], vector_size=100, window=2, min_count=2, min_n=1, max_n=2, sg=1)

# A large window size allows the model to look at more context

print(fast_n)

FastText<vocab=67, vector_size=100, alpha=0.025>


In [36]:
# Similar words using Fast Text

fast_n.wv.most_similar(positive=['human'], topn=10)

[('chapman', 0.9678015112876892),
 ('information', 0.962955117225647),
 ('communication', 0.9569567441940308),
 ('marketing', 0.9505852460861206),
 ('management', 0.9505007863044739),
 ('coordinator', 0.9504880309104919),
 ('senior', 0.94764643907547),
 ('manager', 0.9463614225387573),
 ('aspiring', 0.9455015659332275),
 ('english', 0.9403043389320374)]

In [37]:
# Job_title to Vectors using FasttText Model

ft_mean= [get_mean_vector(fast_n,i) for i in Data_FastText.job_title]

fastT_mean=pd.DataFrame(ft_mean)

fastT_mean.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.003896,0.006873,-0.010979,0.003486,-0.002789,-0.003376,0.001189,-0.001066,0.005136,0.000434,...,0.008723,-0.003084,0.004119,0.002535,0.001942,0.004532,-0.006140,0.002993,-0.002063,-0.000105
1,-0.006597,0.007489,-0.011625,0.003552,-0.003070,-0.003396,0.002458,-0.000059,0.006260,0.000961,...,0.009276,-0.003348,0.006073,0.002382,0.002381,0.004785,-0.005924,0.002485,-0.003362,0.000035
2,-0.006791,0.007601,-0.012338,0.002884,-0.002167,-0.004100,0.000654,-0.000045,0.006197,0.000242,...,0.009625,-0.003667,0.005863,0.002772,0.003214,0.006968,-0.006235,0.002630,-0.003245,-0.000507
3,-0.006577,0.007967,-0.013006,0.002669,-0.002314,-0.004006,0.003616,-0.000570,0.006656,-0.000872,...,0.009490,-0.003293,0.006551,0.003259,0.002938,0.004796,-0.007104,0.003625,-0.002344,-0.000497
4,-0.005842,0.006454,-0.010355,0.003123,-0.003086,-0.003537,0.002135,-0.000331,0.004739,0.000866,...,0.009224,-0.003087,0.005941,0.000973,0.002310,0.004737,-0.005045,0.002923,-0.002246,0.000787


In [38]:
fast_text_vectors=fastT_mean.to_numpy()
print('fast_text_vectors_Shape: ',fast_text_vectors.shape)

fast_text_vectors_Shape:  (104, 100)


In [39]:
Query= "Aspiring human resources"

Top_n(Query, fast_n,fast_text_vectors,Data_FastText,10)

,id,job_title,location,connection,fit
32,33,"[aspiring, human, resource, professional]","[raleighdurham, north, carolina, area]",44,[0.9921238]
75,76,"[aspiring, human, resource, professional, pass...","[new, york, new, york]",212,[0.9921238]
72,73,"[aspiring, human, resource, manager, seeking, ...","[houston, texas, area]",7,[0.99131775]
59,60,"[aspiring, human, resource, specialist]","[greater, new, york, city, area]",1,[0.9911752]
81,82,"[aspiring, human, resource, professional, ener...","[austin, texas, area]",174,[0.99098694]
65,66,"[experienced, retail, manager, aspiring, human...","[austin, texas, area]",57,[0.99053544]
27,28,"[seeking, human, resource, opportunity]","[chicago, illinois]",390,[0.99011165]
93,94,"[seeking, human, resource, opportunity, open, ...","[amerika, birleşik, devletleri]",415,[0.99011165]
52,53,"[seeking, human, resource, hris, generalist, p...","[greater, philadelphia, area]",500+,[0.9900405]
66,67,"[human, resource, staffing, recruiting, profes...","[jackson, mississippi, area]",500+,[0.9899471]


# GloVE

In [40]:
Data_glove=orig_data.copy()

In [41]:
embeddings = {}
with open(r"C:\Users\97158\Downloads\glove.6B\glove.6B.100d.txt", encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = [float(val) for val in values[1:]]
        embeddings[word] = vector
        #coefs = np.asarray(values[1:], dtype='float32')
        #embedding_index[word] = coefs

    print('Found %s word vectors.' % len(embeddings))
    glove_embed=embeddings

Found 400000 word vectors.


In [42]:
import numpy as np

def sentence_embedding(sentence, embeddings):
    words = sentence
    vectorized_words = [embeddings.get(word, np.zeros(100)) for word in words]
    sentence_emb = np.mean(vectorized_words, axis=0)
    return sentence_emb

In [43]:
import numpy as np

def query_embedding(Query, embeddings):
    words = Query.split()
    vectorized_words = [embeddings.get(word, np.zeros(100)) for word in words]
    sentence_emb = np.mean(vectorized_words, axis=0)
    return sentence_emb

In [44]:
Query= "Aspiring human resources"


def Glove_Similarity_Score_Cal(df,Query,embedding):                     
    score=[]
    for index,row in df.iterrows():
        sentence=row['job_title']

        # Compute embeddings
        sentence_emb = sentence_embedding(sentence, embedding)
        sentence_emb = sentence_emb.reshape(1,-1) 
        
        Query_emb = query_embedding(Query, embedding)
        Query_emb = Query_emb.reshape(1,-1) 
        
                                        
        #Calculating cosine_similarity      
        Similarity_score=cosine_similarity(sentence_emb,Query_emb)
        score.append(Similarity_score[0][0])
    
    df['fit']=score
    df=df.sort_values(by='fit', ascending=False)
    GloVe_df=df.drop_duplicates(subset=['job_title'])

    return GloVe_df.head(15)

In [45]:
Glove_Results=Glove_Similarity_Score_Cal(data,Query,glove_embed)
Glove_Results.iloc[:10]

,id,job_title,location,connection,fit
29,30,"[seeking, human, resource, opportunity]","[chicago, illinois]",390,0.887072
87,88,"[human, resource, management, major]","[milpitas, california]",18,0.880439
98,99,"[seeking, human, resource, position]","[la, vega, nevada, area]",48,0.879327
73,74,"[human, resource, professional]","[greater, boston, area]",16,0.868421
72,73,"[aspiring, human, resource, manager, seeking, ...","[houston, texas, area]",7,0.855437
52,53,"[seeking, human, resource, hris, generalist, p...","[greater, philadelphia, area]",500+,0.851628
76,77,"[human, resource, conflict, management, policy...","[dallasfort, worth, area]",409,0.843459
100,101,"[human, resource, generalist, loparex]","[raleighdurham, north, carolina, area]",500+,0.837878
88,89,"[director, human, resource, ey]","[greater, atlanta, area]",349,0.837395
80,81,"[senior, human, resource, business, partner, h...","[chattanooga, tennessee, area]",455,0.821940


# BERT

In [46]:
Data_Bert=orig_data.copy()

In [47]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers.util import cos_sim
import torch

In [48]:
model_name="bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(model_name)
bert_model=AutoModel.from_pretrained(model_name)


In [49]:
def tokenize_sent(sentences):
    return tokenizer(sentences,padding=True, truncation=True,return_tensors='pt')  #'pt' here is pytorch format

In [50]:
def compute_embeddings(tokenized_sent):
    with torch.no_grad():
        outputs=bert_model(**tokenized_sent)   #unpacking tokenized_sent because this is a dictionary
        embeddings=outputs.last_hidden_state.mean(dim=1)
    return embeddings

In [51]:
def compute_embeddings(tokenized_sent):
    with torch.no_grad():
        outputs=bert_model(**tokenized_sent)   #unpacking tokenized_sent because this is a dictionary
        embeddings=outputs.last_hidden_state.mean(dim=1)   #mean_pooling

    return embeddings

In [52]:
def Bert_Similarity_score_Cal(df,Query,Top_n):                     
    score=[]
    for index,row in df.iterrows():
        sentence=row['job_title']

        #Tokenizing and computing embeddings for raw data
        tokenized_sent=tokenize_sent(sentence)
        sentence_embed=compute_embeddings(tokenized_sent)

        #Tokenizing and computing embedding for input query_mean
        tokenized_query=tokenize_sent(Query)
        query_embed=compute_embeddings(tokenized_query)
                                        
        #Calculating cosine_similarity      
        Similarity_score=cosine_similarity(sentence_embed,query_embed)
        score.append(Similarity_score[0][0])
         
    df['fit']=score
    df=df.sort_values(by='fit', ascending=False)
    df=df.drop_duplicates(subset=['job_title'])
    df=df.iloc[:Top_n]
    return df.head(15)

In [53]:
Query= "Aspiring Human Resources"

In [54]:
Bert_Similarity_score_Cal(Data_Bert,Query,10)

,id,job_title,location,connection,fit
59,60,Aspiring Human Resources Specialist,Greater New York City Area,1,0.905480
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.902632
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.823453
98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,0.815410
73,74,Human Resources Professional,Greater Boston Area,16,0.814312
50,51,HR Senior Specialist,San Francisco Bay Area,500+,0.804810
65,66,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57,0.766706
26,27,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,0.756199
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,0.752288
67,68,Human Resources Specialist at Luxottica,Greater New York City Area,500+,0.749704


# Alternative Bert Method

In [55]:
Data_Bert2=orig_data.copy()

In [56]:
model_name="bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(model_name)
Bert_model=AutoModel.from_pretrained(model_name)

In [57]:
def string_to_bert_embeddings(string):
    toketoken_to_encode=tokenizer.encode_plus(string,add_special_tokens=True, return_tensors='pt')
    output=Bert_model(**toketoken_to_encode)
    mean_pool=torch.mean(output.last_hidden_state,dim=1)
    mean_pool=mean_pool.detach().numpy()
    return mean_pool

In [58]:
Query= "Aspiring Human Resources"

# Converting Query and job_titles into Bert embeddings
Query_mean_pool=string_to_bert_embeddings(Query)
Data_Bert2['fit']=Data_Bert2['job_title'].apply(lambda x: string_to_bert_embeddings(x))


In [59]:
def Sim_Score_Cal(df_with_embed,Query_embed):
    df_with_embed['sim_score']=df_with_embed['fit'].apply(lambda x: cosine_similarity(Query_embed,x))
    df_with_embed['sim_score']=df_with_embed['sim_score'].apply(lambda x: x[0][0])
    df_with_embed=df_with_embed.sort_values(by='sim_score',ascending=False)
    df_with_embed=df_with_embed.drop_duplicates(subset='job_title')
    df_with_embed=df_with_embed.reset_index().drop(['id','index'],axis=1)
 
    return df_with_embed

In [60]:
Bert_Results=Sim_Score_Cal(Data_Bert2,Query_mean_pool)
Bert_Results.iloc[:10]

,job_title,location,connection,fit,sim_score
0,Aspiring Human Resources Specialist,Greater New York City Area,1,"[[-0.06974506, -0.06908064, 0.06281222, 0.1142...",0.905480
1,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,"[[-0.017994473, -0.120408334, -0.052445814, 0....",0.902632
2,Seeking Human Resources Opportunities,"Chicago, Illinois",390,"[[-0.15450734, 0.1367554, -0.0969936, 0.158022...",0.823453
3,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,"[[-0.0395243, 0.06603463, -0.13802424, 0.08560...",0.815410
4,Human Resources Professional,Greater Boston Area,16,"[[-0.08023474, 0.13505873, -0.21095267, 0.1579...",0.814312
5,HR Senior Specialist,San Francisco Bay Area,500+,"[[-0.062803745, -0.15042305, 0.19895987, -0.23...",0.804810
6,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57,"[[-0.26336306, 0.08374588, 0.041640982, 0.0719...",0.766706
7,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,"[[-0.14394718, 0.14305301, 0.120392, 0.0114721...",0.756199
8,People Development Coordinator at Ryan,"Denton, Texas",500+,"[[-0.3592906, -0.14920676, -0.21835914, -0.006...",0.752288
9,Human Resources Specialist at Luxottica,Greater New York City Area,500+,"[[-0.18072551, -0.081646845, 0.08086378, 0.209...",0.749704


# SBERT

In [61]:
Data_SBert=orig_data.copy()

In [62]:
from sentence_transformers import SentenceTransformer, util
sbert_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [63]:
def SBERT_Similarity_Score_Cal(df,Query,model,Top_n):                     
    score=[]
    for index,row in df.iterrows():
        sentence=row['job_title']

        #Tokenizing and computing embeddings for raw data
        sentence_embed=model.encode(sentence)
        sentence_embed=sentence_embed.reshape(1,-1)
        
        #Tokenizing and computing embedding for input query_mean
        query_embed=model.encode(Query)
        query_embed=query_embed.reshape(1,-1)
      
                               
        #Calculating cosine_similarity      
        Similarity_score=cosine_similarity(sentence_embed,query_embed)
        score.append(Similarity_score[0][0])
    
    print('this is score scliced: ',score)    
    df['fit']=score
    df=df.sort_values(by='fit', ascending=False)
    df=df.drop_duplicates(subset=['job_title'])
    df=df.iloc[:Top_n]
    return df

In [64]:
Query= "Aspiring Human Resources"

SBERT_Similarity_Score_Cal(Data_SBert,Query,sbert_model,10)

this is score scliced:  [0.39507496, 0.13104543, 0.91386473, 0.33009738, 0.1457411, 0.87727505, 0.7171319, 0.4251589, 0.7171319, 0.72883046, 0.24243638, 0.29137367, 0.501158, 0.39507496, 0.39507496, 0.13104543, 0.91386473, 0.33009738, 0.39507496, 0.13104543, 0.91386473, 0.33009738, 0.1457411, 0.87727505, 0.7171319, 0.4251589, 0.65595484, 0.8801758, 0.65595484, 0.8801758, 0.39507496, 0.13104543, 0.91386473, 0.33009738, 0.1457411, 0.87727505, 0.7171319, 0.4251589, 0.7171319, 0.72883046, 0.24243638, 0.29137367, 0.501158, 0.39507496, 0.13104543, 0.91386473, 0.33009738, 0.1457411, 0.87727505, 0.7171319, 0.4251589, 0.7171319, 0.72883046, 0.24243638, 0.29137367, 0.501158, 0.39507496, 0.91386473, 0.33009738, 0.87727505, 0.4251589, 0.72883046, 0.24243638, 0.29137367, 0.501158, 0.607157, 0.67684466, 0.5892706, 0.5250673, 0.53841686, 0.6262528, 0.6301339, 0.64597166, 0.7967472, 0.35173523, 0.8035265, 0.49841937, 0.595207, 0.532621, 0.32848972, 0.39834946, 0.7833301, 0.31246418, 0.599666, 0.124389

,id,job_title,location,connection,fit
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.913865
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.880176
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,0.877275
98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,0.861658
75,76,Aspiring Human Resources Professional | Passio...,"New York, New York",212,0.803527
73,74,Human Resources Professional,Greater Boston Area,16,0.796747
81,82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,0.783330
39,40,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,0.728830
51,52,Student at Humber College and Aspiring Human R...,Kanada,61,0.717132
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,0.710260


# Alternative SBERT Method:

In [65]:
Data_SBert2=orig_data.copy()

In [66]:
import transformers

In [67]:
model_name= 'sentence-transformers/multi-qa-MiniLM-L6-cos-v1' 
SBert_model=transformers.AutoModel.from_pretrained(model_name)
Tokenizer=transformers.AutoTokenizer.from_pretrained(model_name)

In [68]:
# Creating def function to convert string into embedding

def  string_to_sbert_embedding(string):
    tokens_to_encode=Tokenizer.encode_plus(string, add_special_tokens=True, return_tensors='pt')
    output=SBert_model(**tokens_to_encode)
    #print('last_hidden_state: ',output.last_hidden_state.shape)
    mean_pool=torch.mean(output.last_hidden_state,dim=1)
    mean_pool=mean_pool.detach().numpy()
    #print('mean_pool: ',mean_pool.shape)
    return mean_pool

In [69]:
Query= "Aspiring Human Resources"

#Converting Query and job_titles into embeddings 
Query_mean_pool=string_to_sbert_embedding(Query)
Data_SBert2['fit']=Data_SBert2['job_title'].apply(lambda x: string_to_sbert_embedding(str(x)))

In [70]:
SBert2_results=Sim_Score_Cal(Data_SBert2,Query_mean_pool)
SBert2_results.iloc[:10]

,job_title,location,connection,fit,sim_score
0,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,"[[-0.20435591, 0.087272234, -0.17087007, 0.368...",0.913865
1,Seeking Human Resources Opportunities,"Chicago, Illinois",390,"[[-0.10797528, -0.11336738, 0.088795684, 0.393...",0.880176
2,Aspiring Human Resources Specialist,Greater New York City Area,1,"[[-0.3970478, 0.31730077, -0.05147621, 0.55939...",0.877275
3,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,"[[-0.16889131, -0.013320466, -0.12300869, 0.19...",0.861658
4,Aspiring Human Resources Professional | Passio...,"New York, New York",212,"[[0.07920582, 0.051878676, -0.0073703625, 0.38...",0.803527
5,Human Resources Professional,Greater Boston Area,16,"[[-0.3385644, 0.22772124, -0.28680634, 0.15234...",0.796747
6,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,"[[-0.04780766, 0.314151, -0.32594964, 0.299894...",0.783330
7,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,"[[-0.15483312, -0.1335427, -0.10757227, 0.0100...",0.728830
8,Student at Humber College and Aspiring Human R...,Kanada,61,"[[0.02680271, 0.19020161, 0.03406019, 0.153044...",0.717132
9,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,"[[-0.11864401, -0.0032189891, 0.11862442, 0.20...",0.710260


# Reranking Algorithms Using SBERT

In [71]:
# Method1:
# New keyword embedding= Embedding(Query + Selected Query)

# --OR--

# Method2:
# New keyword embedding= Mean(Embedding(Query) + Embedding(Selected Query))

In [72]:
#Using Method 1
def Sum_New_keyword_embed(Selected_Query_Index,Query,model_results):
    
    New_keyword_embed=string_to_sbert_embedding(Query+' '+model_results.iloc[Selected_Query_Index,0])
    return New_keyword_embed

In [73]:
#Using Method 2
def Mean_New_keyword_embed(Selected_Query_Index,Query,model_results):
    Query_embed=string_to_sbert_embedding(Query)
    New_keyword_embed= np.mean([model_results.iloc[Selected_Query_Index,3],Query_embed],axis=0)
    return New_keyword_embed

In [74]:
def Reranker(Selected_Query_Index, Query, model_results, reranking_technique,Top_n):
    
    New_keyword_embed_= reranking_technique(Selected_Query_Index, Query, model_results)
    
    model_results['sim_score']=model_results['fit'].apply(lambda x: cosine_similarity(New_keyword_embed_,x))
    model_results['sim_score']=model_results['sim_score'].apply(lambda x: x[0][0])
    model_results=model_results.sort_values(by='sim_score',ascending=False)
    model_results=model_results.drop_duplicates(subset='job_title')
    model_results=model_results.reset_index().drop(['index','fit'],axis=1)
    model_results=model_results.iloc[:Top_n]
    return model_results

Method 1 Implementation

In [75]:
Query= "Aspiring Human Resources"
Reranker(2,Query, SBert2_results,Sum_New_keyword_embed,10)

,job_title,location,connection,sim_score
0,Aspiring Human Resources Specialist,Greater New York City Area,1,0.981270
1,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.940745
2,Human Resources Professional,Greater Boston Area,16,0.878626
3,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,0.839206
4,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.817944
5,Human Resources Management Major,"Milpitas, California",18,0.801748
6,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,0.800258
7,Aspiring Human Resources Professional | Passio...,"New York, New York",212,0.790256
8,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,0.778322
9,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,0.762304


Method 2 Implementation

In [76]:
Query= "Aspiring Human Resources"
Reranker(2,Query, SBert2_results,Mean_New_keyword_embed,10)

,job_title,location,connection,sim_score
0,Aspiring Human Resources Specialist,Greater New York City Area,1,0.968594
1,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.957737
2,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,0.875270
3,Human Resources Professional,Greater Boston Area,16,0.872061
4,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.866531
5,Aspiring Human Resources Professional | Passio...,"New York, New York",212,0.818397
6,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,0.814361
7,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,0.780296
8,Human Resources Management Major,"Milpitas, California",18,0.768929
9,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,0.756703
